# Implementación preliminar del código

### Descargar paquetes necesarios

In [49]:
#!pip install catboost
#!pip install sweetviz
#!pip install optuna
#!pip install lightgbm
#!pip install scikit-learn
#!pip install xgboost
#!pip install pyreadstat

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/565.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/565.3 kB ? eta -:--:--
     -------------------------------------- 565.3/565.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyreadstat: filename=pyreadstat-1.2.9-cp38-cp38-win_amd64.whl size=319963 sha256=57481a12f9b06739deaee8ee58afc526a116a36b435b68ce32f16f777c63cea6
  Stored in directory: c:\users\josé carlos\appdata\local\pip\cache\wheels\34\62\02\7d2524ef9e4a75c2ae00c41eebb26434b133adb534db354842
Successfully built pyreadstat


In [1]:
# --- Librerías estándar ---
import os  # Rutas de los archivos

# --- Librerías científicas y de análisis ---
import matplotlib.pyplot as plt # Gráficos
import numpy as np  # Funciones matemáticas
import pandas as pd  # Manejo de dataframes
import sweetviz as sviz # EDA

# --- Modelos de clasificación ---
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier, 
                              ExtraTreesClassifier, GradientBoostingClassifier, 
                              RandomForestClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# --- Metricas y evaluación --- 
from sklearn.metrics import (
    accuracy_score, precision_score, f1_score, recall_score
)
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

# --- Optimizador ---
import optuna

# --- Elaboración propia ---
from ProbadorHipotesis import ProbadorHipotesis
from ModeladorDatos import ModeladorDatos
from ImportadorDatos import ImportadorDatos
from OptimizadorExtraTreesOptuna import OptimizadorExtraTreesOptuna

C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lectura de datos

Note que la limpieza y transformación de los datos se efectuó en el script "main_enaho.py"

In [2]:
ruta_actual = os.getcwd()
ruta_datos = os.path.join(ruta_actual, '../..', 'data/base.csv') 
ruta_normalizada = os.path.normpath(ruta_datos) 
base = pd.read_csv(ruta_normalizada)

In [3]:
base.head()

,ZONA,REGION,V1,V2A,V3,V4,V5,V6,V6A,V7A,...,V19,V21,R4A,TamViv,A4,A5,A6,CondMig,NivInst,A22A
0,Urbana,Central,En fila o contigua,Alquiler o cesión,Block o ladrillo,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,2.0,Hombre,45 a 59,Separado/Divorciado/Viudo,No migrante,Secundaria completa,Sí
1,Urbana,Central,En fila o contigua,Alquiler o cesión,Block o ladrillo,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,2.0,Mujer,13 a 17,Soltero(a),Migrante interno,Primaria completa,No
2,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,3.0,Mujer,45 a 59,Casado(a),Migrante externo,Primaria completa,No
3,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,3.0,Hombre,60 a 74,Casado(a),Migrante externo,Primaria completa,Sí
4,Urbana,Central,En fila o contigua,Propiedad,Otro,Lámina de metal o zinc,Sí,"Mosaico, cerámica, terrazo",No,Bueno,...,SÍ,0,No,3.0,Mujer,25 a 34,Soltero(a),No migrante,Superior,Sí


## Se evaluan los modelos

In [6]:
models = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTree": DecisionTreeClassifier(),
    #"SVM": SVC(probability=True),
    "RandomForest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(),
    "KNN": KNeighborsClassifier(),
    "NeuralNetwork": MLPClassifier(),
    "LDA": LinearDiscriminantAnalysis(),
    "CatBoost": CatBoostClassifier(),
    "LightGBM": LGBMClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "NaiveBayes": GaussianNB(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "Bagging": BaggingClassifier(base_estimator=DecisionTreeClassifier()),
    "ExtraTrees": ExtraTreesClassifier()
}
modelador = ModeladorDatos(df = base, var_objetivo = 'V21', modelos = models)

#### evaluacion de los modelos con el modelador

In [7]:
results_df = modelador.evaluar_modelos()

Training LogisticRegression...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training DecisionTree...
Training RandomForest...
Training XGBoost...
Training KNN...
Training NeuralNetwork...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training LDA...
Training CatBoost...
Learning rate set to 0.036085
0:	learn: 0.6656510	total: 155ms	remaining: 2m 35s
1:	learn: 0.6406955	total: 176ms	remaining: 1m 28s
2:	learn: 0.6167951	total: 197ms	remaining: 1m 5s
3:	learn: 0.5962398	total: 228ms	remaining: 56.7s
4:	learn: 0.5769672	total: 254ms	remaining: 50.5s
5:	learn: 0.5593462	total: 274ms	remaining: 45.4s
6:	learn: 0.5440616	total: 294ms	remaining: 41.7s
7:	learn: 0.5288220	total: 313ms	remaining: 38.8s
8:	learn: 0.5159265	total: 333ms	remaining: 36.7s
9:	learn: 0.5071103	total: 353ms	remaining: 34.9s
10:	learn: 0.4965033	total: 374ms	remaining: 33.7s
11:	learn: 0.4857068	total: 395ms	remaining: 32.5s
12:	learn: 0.4778379	total: 413ms	remaining: 31.4s
13:	learn: 0.4697625	total: 435ms	remaining: 30.6s
14:	learn: 0.4619196	total: 455ms	remaining: 29.9s
15:	learn: 0.4562249	total: 476ms	remaining: 29.3s
16:	learn: 0.4514157	total: 497ms	remaining: 28.8s
17:	learn: 0.4463129	total: 517ms	remaining: 28.2s
18:	learn: 0.4409676	to

C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Training Bagging...


C:\Users\José Carlos\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Training ExtraTrees...


In [8]:
# Guardar en Excel
ruta_resultado_modelos = os.path.join(ruta_actual, '../..', 'res/resultados_modelos.xlsx')
ruta_resultado_modelos_norm = os.path.normpath(ruta_resultado_modelos)
results_df.drop(columns=["Predictions"]).to_excel(ruta_resultado_modelos_norm, index=False)
print(results_df)

                 Model  Accuracy   ROC AUC      Gini  Precision    Recall  \
0   LogisticRegression  0.839445  0.851219  0.702438   0.666337  0.412377   
1         DecisionTree  0.916749  0.871201  0.742401   0.793758  0.794730   
2         RandomForest  0.943756  0.979797  0.959594   0.955178  0.757353   
3              XGBoost  0.900768  0.943868  0.887736   0.822844  0.648897   
4                  KNN  0.886397  0.913744  0.827487   0.765011  0.632353   
5        NeuralNetwork  0.904361  0.944317  0.888634   0.744596  0.802083   
6                  LDA  0.834490  0.836813  0.673626   0.649495  0.393995   
7             CatBoost  0.895937  0.936683  0.873365   0.836735  0.602941   
8             LightGBM  0.878097  0.915967  0.831935   0.780763  0.552083   
9             AdaBoost  0.837587  0.845573  0.691146   0.667712  0.391544   
10    GradientBoosting  0.846259  0.865019  0.730038   0.714599  0.398897   
11          NaiveBayes  0.596011  0.807905  0.615811   0.319361  0.882353   

Se considera que el modelo de ExtraTrees es el más adecuado dado a su alto score tanto en precisión como en recall, por lo que se va a utilizar para la optimización de parámetros

#### Se crean los conjuntos de prueba y entrenamiento

In [13]:
X_train, X_test, y_train, y_test = train_test_split(base.drop(['V21'], axis = 1),
                                                    base['V21'],
                                                    test_size = 0.3,
                                                    random_state = 2
                                                    )

### Se crea la funcion a optimizar

In [9]:
optimizador = OptimizadorExtraTreesOptuna(base, "V21", 50)

In [10]:
optimizador.optimizar()

[I 2025-06-07 16:29:24,641] A new study created in memory with name: no-name-2db42d35-7090-4f7e-89a5-5502ed45d649
[I 2025-06-07 16:29:29,108] Trial 0 finished with value: 0.7568378133823663 and parameters: {'n_estimators': 196, 'max_depth': 22, 'min_samples_split': 39, 'min_samples_leaf': 13, 'max_features': 0.5543984942271671, 'max_leaf_nodes': 134}. Best is trial 0 with value: 0.7568378133823663.
[I 2025-06-07 16:29:59,017] Trial 1 finished with value: 0.757163240694168 and parameters: {'n_estimators': 489, 'max_depth': 18, 'min_samples_split': 43, 'min_samples_leaf': 4, 'max_features': 0.4568652261804602, 'max_leaf_nodes': 115}. Best is trial 1 with value: 0.757163240694168.
[I 2025-06-07 16:30:09,980] Trial 2 finished with value: 0.6838986496461618 and parameters: {'n_estimators': 168, 'max_depth': 5, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 0.5694243005414459, 'max_leaf_nodes': 124}. Best is trial 1 with value: 0.757163240694168.
[I 2025-06-07 16:30:51,273] T

Mejores hiperparámetros encontrados:
{'n_estimators': 204, 'max_depth': 40, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.5965759182136633, 'max_leaf_nodes': 265}
Mejor F1 (macro): 0.7986682968037271


"Parámetros: {'n_estimators': 204, 'max_depth': 40, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.5965759182136633, 'max_leaf_nodes': 265}\nScore: 0.7986682968037271"

In [11]:
optimizador.entrenar_modelo_final()

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['V18A1']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['ZONA', 'REGION', 'V1',
                                                   'V2A', 'V3', 'V4', 'V5',
                                                   'V6', 'V6A', 'V7A', 'V7B',
                                                   'V7C', 'EFI', 'V8', 'V11',
                                                   'V12', 'V13A', 'V14A1',
                                                   'V15', 'V16', 'V17A',
                                                   'V17B1', 'V17B2', 'V17B3',
                                                   'V17B4', 'V17B5', 'V18C',
                                                   'V18D', 'V18E', 'V18F', ...])],
                                   verbose_feature_names_out=False)),
                ('classifier',
                 ExtraTreesClassifier(max_depth=40,
                                      max_features=0.5965759182136633,
                                      max_leaf_nodes=265, min_samples_split=5,
                                      n_estimators=204, n_jobs=-1,
                                      random_state=42))])

#### Se calculan predicciones con distintas probabilidades y se generan los resultados

In [12]:
optimizador.evaluar_modelo_final()

,Accuracy,Precisión,Recall,F1
0,0.202180,0.202180,1.000000,0.336356
1,0.729559,0.419703,0.882353,0.568833
2,0.843038,0.580503,0.806373,0.675045
3,0.879212,0.691322,0.727328,0.708868
4,0.889742,0.770803,0.647059,0.703531
5,0.890610,0.831709,0.575368,0.680188
6,0.884044,0.879085,0.494485,0.632941
7,0.871531,0.933042,0.392770,0.552824
8,0.844648,0.979695,0.236520,0.381046
9,0.801784,1.000000,0.019608,0.038462


A partir de estos resultados, se observa que el desempeño global del modelo (según la métrica F1) es óptimo cuando se escoge un umbral de probabilidad alrededor de 30%, lo que representa el mejor balance entre precisión y sensibilidad para predecir la tenencia de bono de vivienda.